<font size="3" color="brown">Author : Donna Tinge</font><br>
Find me on: [LinkedIn](https://www.linkedin.com/in/donnatinge/) [GitHub](https://github.com/donnatinge/)


# Install and import the required packages
## pandas, pandasql, xlrd, matplotlib

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26818 sha256=3965a12c1fe6abb47b8100834d40d7da5d553364bf11b3f1417dea531fa06492
  Stored in directory: /Users/keerthipro/Library/Caches/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt

# Import the datasets

In [5]:
#Importing Opportunities dataset
opp_data = pd.read_excel("https://github.com/donnatinge/Twilio/blob/main/Core%20Analytics%20Take%20Home%20Challenge%20Data%20Set.xlsx?raw=true", 
                             sheet_name="Opportunities Data")
opp_data.head()

,Deal Close Date,Region,Sub Region,Segment,Deal Start Date,Opportunity ID,Pipeline Stage,Pipeline Sub Stage,Status,eARR
0,2020-06-19,NA Mid-Market,NorthWest,Mid-Market,2020-01-29,ABCDEF0001SaFZ7QAN,Omitted,Closed Lost,SQL Accepted,18000.00
1,2020-02-05,APAC,ANZ,Growth,2019-07-15,ABCDEF0001IC0qlQAD,Omitted,Closed Lost,SQL Accepted,4800.00
2,2020-01-02,NA Growth,West,Growth Strategic,2019-07-31,ABCDEF0001JSWfaQAH,Omitted,Closed Lost,SQL Accepted,36000.00
3,2020-06-29,EMEA,UKI,Growth,2020-05-26,ABCDEF0001SxR24QAF,Closed,Closed Won,SQL Accepted,11268.48
4,2020-06-25,NA Growth,NorthEast,Growth,2019-03-17,ABCDEF0001AqExyQAF,Closed,Closed Won,SQL Accepted,3662.28


In [6]:
#Importing Reps dataset
rep_data = pd.read_excel("https://github.com/donnatinge/Twilio/blob/main/Core%20Analytics%20Take%20Home%20Challenge%20Data%20Set.xlsx?raw=true", 
                            sheet_name="Rep Data")
rep_data.head()

,Period,Rep ID,Region,Sub Region,Segment,RSE
0,2020-01-01,12345600004RbBLQA0,NA Growth,SouthEast,Growth,1.0
1,2020-02-01,12345600004RbBLQA0,NA Growth,SouthEast,Growth,1.0
2,2020-03-01,12345600004RbBLQA0,NA Growth,SouthEast,Growth,1.0
3,2020-04-01,12345600004RbBLQA0,NA Growth,SouthEast,Growth,1.0
4,2020-05-01,12345600004RbBLQA0,NA Growth,SouthEast,Growth,1.0


# Questions Answered using SQL

### Q1. What’s the total RSE by each region and segment?

In [156]:
q1 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------

SELECT 
--    DATE(period),
    STRFTIME('%Y-%m', period) AS month,
    region,
    segment,
    IFNULL(SUM(rse),0) AS "Total RSE"
FROM rep_data
GROUP BY STRFTIME('%Y-%m', period), region, segment

---------------------WRITE QUERY ABOVE---------------------
""")

q1

,month,Region,Segment,Total RSE
0,2020-01,APAC,Enterprise,6.7667
1,2020-01,APAC,Enterprise Strategic,0.6333
2,2020-01,APAC,Growth,4.3376
3,2020-01,APAC,Growth Strategic,1.2166
4,2020-01,APAC,Mid-Market,4.0000
...,...,...,...,...
140,2020-06,NA Mid-Market,Mid-Market Strategic,6.4169
141,2020-06,NA West,Enterprise,28.5334
142,2020-06,NA West,Enterprise Strategic,4.1133
143,2020-06,NA West,ISV,4.6500


### Explanation
1. Included the Month in calculating the Total RSEs
2. Blank RSEs are converted to zero using IFNULL()

### Q2. What’s the average life cycle by each region?

In [158]:
q2 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------

SELECT 
    region,
    CAST(AVG(julianday("deal close date") - julianday("deal start date")) AS INT) "Average Life Cycle (days)"
FROM opp_data
WHERE 
    "deal start date" IS NOT NULL
GROUP BY region
ORDER BY "Average Life Cycle (days)" DESC
    

---------------------WRITE QUERY ABOVE---------------------
""")

q2

,Region,Average Life Cycle (days)
0,NA West,170
1,NA East,152
2,APAC,142
3,NA Mid-Market,136
4,LATAM,134
5,NA Growth,131
6,EMEA,110


### Explanation
1. Excluded the deals which do not have a deal start date
2. Ordered by "Average Life Cycle (days)" to highlight the regions which take longer to close deals

### Q3. What’s the average deal size by each region? Does classifying them into small/big or any more categories help us find better trends?

In [7]:
q3 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------
SELECT 
    region,
    ROUND(AVG(earr),0) as "Average Deal Size"
FROM opp_data
GROUP BY region
ORDER BY "Average Deal Size" Desc
---------------------WRITE QUERY ABOVE---------------------
""")

q3

,Region,Average Deal Size
0,NA West,126654.0
1,NA East,92939.0
2,LATAM,54481.0
3,APAC,44201.0
4,NA Mid-Market,39541.0
5,EMEA,32656.0
6,NA Growth,21688.0


### Q3.1 Does classifying them into small/big or any more categories help us find better trends?

In [171]:
q3_1 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------
SELECT
    region, "sub region",
    ROUND(AVG(earr),0) as "Average Deal Size"
FROM opp_data
GROUP BY region, "sub region"
ORDER BY "Average Deal Size" Desc

---------------------WRITE QUERY ABOVE---------------------
""")

q3_1

,Region,Sub Region,Average Deal Size
0,NA West,SCal,262444.0
1,NA West,Central,139893.0
2,NA East,PubSec,137194.0
3,NA East,New England,129752.0
4,NA East,Verticals,127268.0
5,NA Mid-Market,East,126429.0
6,NA Mid-Market,West,99914.0
7,NA West,MTN,98340.0
8,NA West,Platform,93119.0
9,NA East,South,90186.0


### Explanation
1. First, viewed the average deal size by Region.
2. Included Sub-Region to view the average deal size in more detail. Observed that, NA Mid-Market East and West sub-regions are actually doing better than sub-regions from LATAM and APAC. 
Yes! Classifying this data into detailed categories is helpful in finding more insights.

### Q4. What’s the conversion rate by each region and segment?

### Conversion Rate by number of opportunities closed

In [174]:
q4 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------
WITH cte AS(
SELECT 
    region,
    segment,
    CAST(COUNT("opportunity id") AS FLOAT) AS total_opportunities,
    CAST(COUNT(CASE WHEN status = 'SQL Accepted' THEN "opportunity id" END) AS FLOAT) AS closed_opportunities  
FROM opp_data
GROUP BY 1,2)

SELECT 
    region,
    segment,
    closed_opportunities, 
    total_opportunities,
    ROUND((closed_opportunities/total_opportunities*100),2) AS conversion_rate
FROM cte
ORDER BY conversion_rate DESC

---------------------WRITE QUERY ABOVE---------------------
""")

q4

,region,segment,closed_opportunities,total_opportunities,conversion_rate
0,APAC,Enterprise Strategic,41.0,41.0,100.00
1,NA West,Enterprise Strategic,66.0,68.0,97.06
2,APAC,Enterprise,415.0,433.0,95.84
3,APAC,Growth Strategic,58.0,61.0,95.08
4,NA Mid-Market,Mid-Market Strategic,128.0,135.0,94.81
5,NA Growth,Growth Strategic,964.0,1020.0,94.51
6,EMEA,Enterprise Strategic,17.0,18.0,94.44
7,NA East,Enterprise Strategic,75.0,80.0,93.75
8,EMEA,Growth Strategic,397.0,426.0,93.19
9,EMEA,Mid-Market,574.0,620.0,92.58


### Conversion Rate by eARR

In [175]:
q4_1 = ps.sqldf("""
---------------------WRITE QUERY BELOW---------------------
WITH cte AS(
SELECT 
    region,
    segment,
    CAST(SUM("eArr") AS FLOAT) AS total_opportunities,
    CAST(SUM(CASE WHEN status = 'SQL Accepted' THEN "eArr" END) AS FLOAT) AS closed_opportunities  
FROM opp_data
GROUP BY 1,2)

SELECT 
    region,
    segment,
    closed_opportunities, 
    total_opportunities,
    ROUND((closed_opportunities/total_opportunities*100),2) AS conversion_rate
FROM cte
ORDER BY conversion_rate DESC


---------------------WRITE QUERY ABOVE---------------------
""")

q4_1

,region,segment,closed_opportunities,total_opportunities,conversion_rate
0,APAC,Enterprise Strategic,7908426.71,7908426.71,100.00
1,NA West,Enterprise Strategic,32171200.76,32191200.76,99.94
2,NA East,Enterprise Strategic,27625787.36,27703227.36,99.72
3,NA Mid-Market,Mid-Market Strategic,14341565.08,14416365.08,99.48
4,NA West,ISV,11062244.78,11267445.62,98.18
5,APAC,Enterprise,47583596.60,48481746.56,98.15
6,NA Growth,Growth Strategic,46841460.99,47922515.03,97.74
7,EMEA,Growth Strategic,6754558.73,6925177.89,97.54
8,NA West,Enterprise,85176870.29,87374681.41,97.48
9,EMEA,Mid-Market,25757375.85,26450325.42,97.38


### Explanation
Calculated Conversion Rate by 'Number of Opportunities' and 'Total eARR'

Some interesting observations in comparing the above two methods of calculating the conversion rate.
'NA East Enterprise Strategic' ranked number 8 when viewed by the number of opportunities converted but rose up to rank 3 when looked at conversion rate by 'Total eARR'. So, it actually did very well.

LATAM did not rank in top 15s when looked by either of the categories. This prompts that some action to be taken here.


## ***Please see the attached Tableau Workbook for insightful trends and visualizations***